In [24]:
%reload_ext autoreload
%autoreload 2

import vtk
from vtk.util import numpy_support

import numpy as np
import bloscpack as bp

import sys
import os
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from annotation_utilities import *
from metadata import *

import time

import matplotlib.pyplot as plt
%matplotlib inline

from vis3d_utilities import *

In [14]:
volume_dir

'/home/yuncong/CSHL_volumes2/'

In [46]:
stack = 'MD594'

In [47]:
label_to_name = {}
name_to_label = {}

with open(os.path.join(volume_dir, stack, stack+'_down32_annotationVolume_nameToLabel.txt'), 'r') as f:
    for line in f.readlines():
        name_s, label = line.split()
        label_to_name[int(label)] = name_s
        name_to_label[name_s] = int(label)
        
with open(os.path.join(volume_dir, stack, stack+'_down32_annotationVolume_bbox.txt'), 'r') as f:
    vol_xmin, vol_xmax, vol_ymin, vol_ymax, vol_zmin, vol_zmax = map(int, f.readline().strip().split())

In [48]:
colors = {name_u: np.array(random_colors(1)[0])/255.
          for name_u in set([convert_name_to_unsided(name_s) for name_s in name_to_label.keys()])}

In [49]:
# with open('colors.txt', 'r') as f:
#     colors = {l[0]: np.r_[float(l[1]), float(l[2]), float(l[3])] for l in map(lambda x: x.split(), f.readlines())}

In [50]:
volume = bp.unpack_ndarray_file(os.path.join(volume_dir, '%(stack)s/%(stack)s_down32_annotationVolume.bp' % {'stack':stack})).astype(np.int8)
# atlasProjected_volume = fill_sparse_volume(volume_sparse)
# del volume_sparse

In [51]:
# volume_landmark_bboxes = {labels_sided[i-1]: bbox_3d(volume == i) for i in all_indices}

In [52]:
centroid_all_structures = {}

for name_s, label in name_to_label.iteritems():
    
    ys,xs,zs = np.where(volume == label)
    centroid = xs.mean(), ys.mean(), zs.mean()
    
    centroid_all_structures[name_s] = centroid
    
    print name_s, centroid

LRt_R (351.10750433117533, 272.60351509072672, 260.94770675663352)
Sp5C_L (412.89339300958324, 228.65436165543719, 103.13021544219426)
Pn_L (113.25797535121976, 297.11902165643863, 141.29756048080336)
7N_R (243.46224620836517, 282.60274395695501, 259.18799681388884)
PBG_L (123.25177237018586, 137.47346234910901, 53.631730216516573)
VLL_L (137.58702281123556, 249.21474227408666, 80.743769424734651)
VLL_R (122.09519986236164, 249.34008143602685, 274.89304352812985)
7N_L (250.61297310204847, 272.39141039417353, 108.90705964625015)
PBG_R (103.63156814449918, 142.93267651888343, 300.26457307060753)
RtTg (136.47166205682674, 271.09686411149823, 178.74744782499371)
Sp5C_R (383.1110695726943, 214.52838423866314, 284.66543609106884)
Pn_R (106.04849950220452, 301.50666097757551, 222.66069312094061)
LRt_L (364.46177054591402, 275.95837953531549, 116.60223316743598)
Sp5O_R (286.58386046072508, 217.73583836858006, 307.10281344410873)
SNR_L (81.321196979653578, 230.58617162411812, 82.269097243747908

In [53]:
canonical_locations = average_location({n: [v] for n, v in centroid_all_structures.iteritems()}, 
                                      [convert_name_to_unsided(n) for n in name_to_label.keys()])

[-0.04010096 -0.03778829  0.99848083] @ [ 205.39148312  209.72426523  181.0122171 ]


In [ ]:
for name_s, label in name_to_label.iteritems():
    
    xmin, xmax, ymin, ymax, zmin, zmax = bbox_3d(volume == label) 
    
    print name_s, xmin, xmax, ymin, ymax, zmin, zmax
    
    vol = (volume[ymin:ymax+1, xmin:xmax+1, zmin:zmax+1] == label).astype(np.float)

    t = time.time()

    polydata = volume_to_polydata(vol, origin=(vol_xmin+xmin,vol_ymin+ymin,vol_zmin+zmin), num_simplify_iter=3, smooth=True)

    sys.stderr.write('time: %.2f\n' % (time.time() - t) ) # ~ 150s for outerContour; 93s for sp5

    save_mesh_stl(polydata, os.path.join(mesh_rootdir, 
                     stack, "%(stack)s_down32_annotationVolume_%(name)s_smoothed.stl" % {'stack': stack, 'name': name_s}))

In [8]:
outerContour_volume = bp.unpack_ndarray_file(os.path.join(volume_dir, '%(stack)s/%(stack)s_down32_outerContourVolume.bp' % {'stack':stack})).astype(np.int8)

with open(os.path.join(volume_dir, stack, stack+'_down32_outerContourVolume_bbox.txt'), 'r') as f:
    oc_xmin, oc_xmax, oc_ymin, oc_ymax, oc_zmin, oc_zmax = map(int, f.readline().strip().split())

In [53]:
t = time.time()

polydata = volume_to_polydata(outerContour_volume, origin=(oc_xmin, oc_ymin, oc_zmin), num_simplify_iter=3, smooth=True)

sys.stderr.write('time: %.2f\n' % (time.time() - t) ) # ~ 150s for outerContour; 93s for sp5

save_mesh_stl(polydata, os.path.join(mesh_rootdir, 
                 stack, "%(stack)s_down32_outerContourVolume_smoothed.stl" % {'stack': stack}))

marching cube: 37.65 seconds
compute surface area: 0.37 seconds
fill point array: 2.37 seconds
fill cell array: 0.07 seconds
mesh_to_polydata: 2.44 seconds
simplify 0 @ 984965: 9.87 seconds
simplify 1 @ 259634: 3.25 seconds
simplify 2 @ 47481: 0.72 seconds
time: 55.11


In [9]:
name_to_label_hindbrain = {name: label for name, label in name_to_label.iteritems() 
                           if convert_name_to_unsided(name) in ['Pn', 'VLL', 'RtTg', 'RMC', '7n', '7N', '5N', '10N', '12N', 'AP', 'LRt', 'Amb', '6N', 'Tz']}

In [10]:
polydata_list = {}

for name_s, label in name_to_label_hindbrain.iteritems():
    
    fn = os.path.join(mesh_rootdir, stack, "%(stack)s_down32_annotationVolume_%(name)s_smoothed.stl" % {'stack': stack, 'name': name_s})

    reader = vtk.vtkSTLReader()
    reader.SetFileName(fn)
    reader.Update()

    polydata = reader.GetOutput()
    polydata_list[name_s] = polydata
    
    
mesh_actors = [actor_mesh(polydata, colors[convert_name_to_unsided(name_s)], wireframe=False, opacity=1.) 
               for name_s, polydata in polydata_list.iteritems()]
# smaller opacity causes erroneous rendering

In [11]:
fn = os.path.join(mesh_rootdir, stack, "%(stack)s_down32_outerContourVolume_smoothed.stl" % {'stack': stack})

reader = vtk.vtkSTLReader()
reader.SetFileName(fn)
reader.Update()

outerContour_polydata = reader.GetOutput()

outerContour_actor = actor_mesh(outerContour_polydata, (1,1,1), opacity=.5, wireframe=False)

In [12]:
launch_vtk(mesh_actors + [outerContour_actor], init_angle='30')

In [67]:
launch_vtk(mesh_actors)

In [ ]:
# Export to rgb PLY file (STL file can not have color info)
# https://github.com/Kitware/VTK/blob/master/IO/PLY/Testing/Python/TestPLYReadWrite.py#L31
# http://www.vtk.org/doc/nightly/html/classvtkPLYWriter.html#aa7f0bdbb2decdc7a7360a890a6c10e8b

for name_s, polydata in polydata_list.iteritems():
    writer = vtk.vtkPLYWriter()
    writer.SetFileName(os.path.join(mesh_rootdir, stack, "%(stack)s_down32_%(name)s_smoothed.ply" % {'stack': stack, 'name':name_s}))
    writer.SetInputData(polydata)
    writer.SetColorModeToUniformColor();
    writer.SetColor((colors[convert_name_to_unsided(name_s)]*255).astype(np.int))
    writer.SetFileTypeToASCII();
    writer.Write()

In [ ]:
writer = vtk.vtkPLYWriter()
writer.SetFileName(os.path.join(mesh_rootdir, stack, "%(stack)s_down32_outerContourVolume_smoothed.ply" % {'stack': stack}))
writer.SetInputData(outerContour_polydata)
writer.SetColorModeToUniformColor();
writer.SetColor(255,255,255)
writer.SetFileTypeToASCII();
writer.Write()